In [ ]:
#FLIGHTRADAR24 DAILY SCRAPPER (this script was trasnformed to exe via pyinstaller and was put on schedule run to collect data)

In [11]:
#Import Libraries for FlightRadar24 Scrapper

In [12]:
import time
import requests
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from fp.fp import FreeProxy

In [13]:
#Change proxy for every "hit", in order to avoid IP ban
def make_me_invisible():
    global proxy
    proxy1 = FreeProxy(timeout=1, rand=True, anonym=False).get()
    proxy2 = 'http://' 
    proxy = {proxy2 : proxy1}
    while len(proxy1) > 27:
        proxy1 = FreeProxy(timeout=1, rand=True, anonym=False).get()
        proxy2 = 'http://' 
        proxy = {proxy2 : proxy1}  
    return proxy

In [14]:
pd.set_option('mode.chained_assignment', None) #settings configuration to avoid Raise Warning due to appending method of csv file
make_me_invisible() #call for proxy change
df_merged = pd.DataFrame() #set empty dataframe
for i in range(10,-10,-1): #JSON Pages that load to the website
    #setting the GET method parameters to request data from the FlightRaar24 API
    URL =  'https://api.flightradar24.com/common/v1/airport.json?code=LHR&plugin[]=&plugin-setting[schedule][mode]=departures&plugin-setting[schedule][timestamp]={}&page='+str(i)+'&limit=100&fleet=&token='
    response = requests.get(URL.format(int(time.time())), headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Safari/605.1.15'})
    data = response.json()
    
    #choose which child of the json we need to put to a dataframe
    df1 = pd.json_normalize(data['result']['response']['airport']['pluginData']['schedule']['departures']['data'])        
    df_merged = pd.concat([df1,df_merged])
    
#data cleansing and re-formatting    
df = df_merged[['flight.identification.number.default','flight.status.text','flight.status.generic.status.text',
                'flight.aircraft.model.code','flight.airport.destination.position.latitude',
                'flight.airport.destination.position.longitude','flight.airport.destination.position.country.code',
                'flight.time.scheduled.departure','flight.time.real.arrival','flight.time.real.departure',
                'flight.time.scheduled.arrival']]
df.rename(columns={'flight.identification.number.default': 'FlightNo','flight.status.generic.status.text':'Status',
                   'flight.time.scheduled.arrival': 'ScheduledTimeArr' , 'flight.status.text' : 'ActualTimeDep',
                   'flight.aircraft.model.code':'AC Model', 'flight.airport.destination.position.latitude' : 'Lat Arr',
                   'flight.airport.destination.position.longitude':'Lon Arr','flight.airport.destination.position.country.code':'Dest Country',
                   'flight.time.real.arrival':'Actual Arr', 'flight.time.real.departure':'Actual Dep',
                   'flight.time.scheduled.departure':'ScheduledTimeDep'
                   }, inplace=True)
df['ScheduledTimeDep'] = pd.to_datetime(df['ScheduledTimeDep'],unit='s')
df['Actual Arr'] = pd.to_datetime(df['Actual Arr'],unit='s')
df['Actual Dep'] = pd.to_datetime(df['Actual Dep'],unit='s')
df['ScheduledTimeArr'] = pd.to_datetime(df['ScheduledTimeArr'],unit='s')
var1 = pd.to_datetime(df['ScheduledTimeArr'] , format='%Y-%m-%d %H:%M:%S').dt.time
var2 = pd.to_datetime(df['Actual Arr'] , format='%Y-%m-%d %H:%M:%S').dt.time.rename('ActualTimeArr')
var3 = pd.to_datetime(df['Actual Dep'] , format='%Y-%m-%d %H:%M:%S').dt.time.rename('ActualTimeDep')
var4 = pd.to_datetime(df['ScheduledTimeDep'] , format='%Y-%m-%d %H:%M:%S').dt.date.rename('Date')
var5 = pd.to_datetime(df['ScheduledTimeDep'] , format='%Y-%m-%d %H:%M:%S').dt.time
df_final = df.drop(['ScheduledTimeDep','Actual Dep','Actual Arr','ScheduledTimeArr'],axis='columns')
df_final= pd.concat((df_final , var1,var2,var3,var4,var5),axis='columns')

In [15]:
# Save the file in CSV format in the current working directory
df_final.to_csv('FlightRadar24.csv', mode='a', header=False)